In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install underthesea
!pip install fairseq
!pip install fastBPE
!pip install tensorflow
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 76.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.5/993.5 kB 58.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.8/657.8 kB 53.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 53.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.5/286.5 kB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.4/112.4 kB 16.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fairseq: filename=fairseq-0.12.2-cp310-cp310-linux_x86_64.whl size=11289851 sha256=544112c78cbd4d44d7dc

In [3]:
import bs4
import time
import re
import requests
import pickle
from underthesea import word_tokenize
from transformers import RobertaForSequenceClassification, RobertaConfig
import torch
import argparse
from fairseq.data.encoders.fastbpe import fastBPE
from fairseq.data import Dictionary
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import os
import json
from tqdm import tqdm

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device "{device}"')


Using device "cuda"


# Encoder

In [8]:
class Encoder:
  def __init__(self):
    # Load BPE encoder
    parser = argparse.ArgumentParser()
    parser.add_argument(
      '--bpe-codes',
      default='/content/drive/MyDrive/pRoBERTa/PhoBERT_base_transformers/bpe.codes',
      required=False,
      type=str,
      help='path to fastBPE BPE'
    )

    args, unknown = parser.parse_known_args()
    self.bpe = fastBPE(args)

    # Load the dictionary
    self.vocab = Dictionary()
    self.vocab.add_from_file('/content/drive/MyDrive/pRoBERTa/PhoBERT_base_transformers/dict.txt')

  def __call__(self, sent1, sent2):
    subwords = ' ' + self.bpe.encode(claim) + ' ' + self.bpe.encode(sent) + ' '
    encoded_sent = self.vocab.encode_line(subwords, append_eos=True, add_if_not_exist=False).long()
    encoded_sent = [encoded_sent]
    encoded_sent = pad_sequences(encoded_sent, maxlen=256, dtype='long', value=1, truncating='post', padding='post')
    mask = [int(token_id != 1) for token_id in encoded_sent[0]]
    mask = [mask]

    return torch.tensor(encoded_sent), torch.tensor(mask)



# Load trained model

In [9]:
model = 'base'

In [10]:
from transformers import RobertaForSequenceClassification, RobertaConfig, AdamW

NUM_CLASSES = 2

# load file config
config = RobertaConfig.from_pretrained(
    f'/content/drive/MyDrive/pRoBERTa/PhoBERT_{model}_transformers/config.json',
    from_tf=False,
    num_labels=NUM_CLASSES,
    output_hidden_states=False
)

# load model
BERT_SA = RobertaForSequenceClassification.from_pretrained(
    f'/content/drive/MyDrive/pRoBERTa/PhoBERT_{model}_transformers/model.bin',
    config=config
)

You are using a model of type bert to instantiate a model of type roberta. This is not supported for all configurations of models and can yield errors.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at /content/drive/MyDrive/pRoBERTa/PhoBERT_base_transformers/model.bin and are newly initialized: ['classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Rationale model

In [11]:
import torch
import torch.nn as nn
from transformers import RobertaModel

class CustomRobertaClassificationHead(nn.Module):
    def __init__(self, hidden_size, num_labels):
        super(CustomRobertaClassificationHead, self).__init__()
        self.dense = nn.Linear(768, 768)
        self.dropout = nn.Dropout(p=0.1)
        self.lstm = nn.LSTM(input_size=hidden_size, hidden_size=256, batch_first=True)
        self.fc1 = nn.Linear(256, 128)
        self.out_proj = nn.Linear(128, num_labels)

    def forward(self, hidden_states):
        x = self.dense(hidden_states)  # Ánh xạ hidden_states qua lớp tuyến tính dense
        x = torch.relu(x)  # Áp dụng hàm kích hoạt ReLU
        x = self.dropout(x)  # Áp dụng dropout
        _, (h_n, _) = self.lstm(x)  # Đưa x qua LSTM và lấy hidden state cuối cùng
        x = self.fc1(h_n[-1])
        x = self.out_proj(x)

        return x

class CustomRobertaForSequenceClassification(nn.Module):
    def __init__(self, model, num_labels):
        super(CustomRobertaForSequenceClassification, self).__init__()

        self.roberta = model
        self.roberta.classifier = CustomRobertaClassificationHead(hidden_size=768, num_labels=num_labels)

    def forward(self, input_ids, token_type_ids, attention_mask):
        outputs = self.roberta(input_ids=input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask)
        return outputs

    def save_pretrained(self, PATH):
       # Create the directory if it doesn't exist
      os.makedirs(PATH, exist_ok=True)

      # Save the model's state_dict to a file
      model_path = os.path.join(PATH, 'model.bin')
      torch.save(self.state_dict(), model_path)


In [12]:
path_rationale_bin = '/content/drive/MyDrive/pRoBERTa/trained_module/rationale_selection/model.bin'
rationale_model = CustomRobertaForSequenceClassification(BERT_SA, 2)
rationale_model.load_state_dict(torch.load(path_rationale_bin))

<All keys matched successfully>

In [13]:
claim = "Theo Adeco Việt Nam, mức lương cao nhất đối với vị trí kỹ sư phần mềm hiện nay là 150 triệu đồng/tháng tại Hà Nội và 160 triệu đồng/tháng tại TP HCM"
sent = "Theo Adeco Việt Nam, mức lương cao nhất đối với vị trí kỹ sư phần mềm hiện nay là 150 triệu đồng/tháng tại Hà Nội và 160 triệu đồng/tháng tại TP HCM"
encoder = Encoder()
token_ids, mask = encoder(preprocess_text(claim), preprocess_text(sent))
token_ids = token_ids.to(device)
mask = mask.to(device)
print(token_ids)
print(mask)

tensor([[   79,  5019, 20792,   350, 10029,     4,   185,  1038,    84,    67,
          3959,    15,   626,  5924,  2199,  5214,   230,  2767,   380,   136,
             8,  2555,   113, 17806,  1701,  7269,    78,    35,  1696,  7657,
             6,  6342,   113, 17806,  1701,  7269,    78,    35,   253,   383,
            79,  5019, 20792,   350, 10029,     4,   185,  1038,    84,    67,
          3959,    15,   626,  5924,  2199,  5214,   230,  2767,   380,   136,
             8,  2555,   113, 17806,  1701,  7269,    78,    35,  1696,  7657,
             6,  6342,   113, 17806,  1701,  7269,    78,    35,   253,   383,
             2,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,  

In [14]:
rationale_model.eval()
rationale_model.cuda()
token_ids, mask
with torch.no_grad():
  outputs = rationale_model(
    token_ids,
    token_type_ids=None,
    attention_mask=mask
  )

  logits = outputs[0]
  logits = logits.detach().to("cpu").numpy()

  print('Label index: ', np.argmax(logits))

Label index:  1


In [15]:
print(logits)

[[-2.3696198  2.3173153]]


# Rationale selection

In [16]:
data_path = '/content/drive/MyDrive/pRoBERTa/data/ise-dsc01-warmup.json'

In [ ]:
# import json

# # Đọc dữ liệu từ file JSON
# with open(data_path) as file:
#     data = json.load(file)

# # Tạo một từ điển để lưu trữ dữ liệu đã chuyển đổi
# converted_data = {}

# # Lặp qua mỗi ID trong data
# for id, item in data.items():
#     data = {}
#     # Lấy giá trị trong key "context"
#     context = item['context']
#     # Phân tách thành mảng các câu
#     sentences = split_into_sentences(context)
#     data["context"] = sentences
#     data["claim"] = tonghop(item["claim"])
#     # Lưu trữ data vào từ điển theo ID tương ứng
#     converted_data[id] = data


In [18]:
import json

# Đọc dữ liệu từ file JSON
with open(data_path) as file:
    data = json.load(file)

# Tạo một từ điển để lưu trữ dữ liệu đã chuyển đổi
converted_data = {}

# Lặp qua mỗi ID trong data
for id, item in data.items():
    data = {}
    data["context"] = item['context']
    data["claim"] = item["claim"]
    # Lưu trữ data vào từ điển theo ID tương ứng
    converted_data[id] = data

In [19]:
rationale_model.cuda()

CustomRobertaForSequenceClassification(
  (roberta): RobertaForSequenceClassification(
    (roberta): RobertaModel(
      (embeddings): RobertaEmbeddings(
        (word_embeddings): Embedding(64001, 768, padding_idx=0)
        (position_embeddings): Embedding(258, 768, padding_idx=0)
        (token_type_embeddings): Embedding(1, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): RobertaEncoder(
        (layer): ModuleList(
          (0-11): 12 x RobertaLayer(
            (attention): RobertaAttention(
              (self): RobertaSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): RobertaSelfOutput

In [20]:
def logits_scale(logits):
    # Khoảng mới
    new_min = -1
    new_max = 1

    # Tính toán phép tỷ lệ tuyến tính
    scaled_x = [(new_max - new_min) * (value - (-5)) / ((5) - (-5)) + new_min for value in logits]
    return scaled_x

In [23]:
from zmq.constants import NULL
encoder = Encoder()

for id, item in tqdm(converted_data.items()):
  evidence_predict = ""
  evidence_predict_max = -1
  idx_evidence = 0
  idx = None
  for sentence in item['context']:
    claim = item['claim']
    sent = preprocess_text(sentence)

    token_ids, mask = encoder(claim, sent)

    token_ids = token_ids.to(device)
    mask = mask.to(device)

    with torch.no_grad():
      outputs = rationale_model(
        token_ids,
        token_type_ids=None,
        attention_mask=mask
      )

      logits = outputs[0]
      logits = logits.detach().cpu().numpy()

      lo_scale = logits_scale(logits[0])
      # Áp dụng hàm softmax
      probabilities = np.exp(lo_scale) / np.sum(np.exp(lo_scale))

      # Tính phần trăm khả năng cho mỗi lớp
      percentages = probabilities * 100

      # In kết quả
      for i, percentage in enumerate(percentages):
          print(f"Lớp {i+1}: {percentage:.2f}%")


      label = np.argmax(logits)
      if (label == 1):
        print("evidence_temp:-- ", sentence)
      # if (label == 1) and (logits[0][label] > evidence_predict_max) and (len(sent.split()) > 3) and (percentages[label] > 76.8):
      if (label == 1) and (logits[0][label] > evidence_predict_max) and (len(sent.split()) > 3):
        evidence_predict = sentence
        evidence_predict_max = logits[0][label]
        idx = idx_evidence
    idx_evidence += 1

  converted_data[id]['evidence'] = evidence_predict if evidence_predict else None
  converted_data[id]['idx_evidence'] = idx
  converted_data[id]['percentages_rationale'] = percentages[label]

  print("\n", id, end = "\n")
  print("claim: --", converted_data[id]['claim'], end = "\n")
  print("evidence_Predicted: --", converted_data[id]['evidence'], end = "\n")



  0%|          | 0/1000 [00:00<?, ?it/s]

Lớp 1: 47.34%
Lớp 2: 52.66%
evidence_temp:--  vòng độc_giả vnexpress bình_chọn sản_phẩm giải_pháp mà yêu_thích
Lớp 1: 77.24%
Lớp 2: 22.76%
Lớp 1: 27.25%
Lớp 2: 72.75%
evidence_temp:--  hai địa_phương hồ_sơ dẫn_đầu bám sát tp hcm điện_biên
Lớp 1: 74.35%
Lớp 2: 25.65%
Lớp 1: 69.59%
Lớp 2: 30.41%
Lớp 1: 72.72%
Lớp 2: 27.28%
Lớp 1: 32.05%
Lớp 2: 67.95%
evidence_temp:--  sản_phẩm ý_tưởng lĩnh_vực công_nghệ thu_hút hồ_sơ y_sinh hóa_sinh nông_nghiệp môi_trường lĩnh_vực


  0%|          | 1/1000 [00:00<06:03,  2.75it/s]

Lớp 1: 69.68%
Lớp 2: 30.32%
Lớp 1: 67.32%
Lớp 2: 32.68%
Lớp 1: 33.79%
Lớp 2: 66.21%
evidence_temp:--  hàng chục tác_giả độ trường thcs thpt
Lớp 1: 75.27%
Lớp 2: 24.73%
Lớp 1: 76.56%
Lớp 2: 23.44%
Lớp 1: 77.60%
Lớp 2: 22.40%
Lớp 1: 50.52%
Lớp 2: 49.48%

 36112
claim: -- hai địa_phương dẫn_đầu hồ_sơ tp hồ chí_minh điện_biên
evidence_Predicted: -- hai địa_phương hồ_sơ dẫn_đầu bám sát tp hcm điện_biên
Lớp 1: 76.95%
Lớp 2: 23.05%
Lớp 1: 70.64%
Lớp 2: 29.36%
Lớp 1: 77.79%
Lớp 2: 22.21%
Lớp 1: 76.14%
Lớp 2: 23.86%
Lớp 1: 78.11%
Lớp 2: 21.89%
Lớp 1: 75.84%
Lớp 2: 24.16%
Lớp 1: 65.70%
Lớp 2: 34.30%
Lớp 1: 77.53%
Lớp 2: 22.47%
Lớp 1: 77.31%
Lớp 2: 22.69%
Lớp 1: 70.66%
Lớp 2: 29.34%
Lớp 1: 76.53%
Lớp 2: 23.47%
Lớp 1: 77.11%
Lớp 2: 22.89%
Lớp 1: 74.45%
Lớp 2: 25.55%


  0%|          | 2/1000 [00:00<05:56,  2.80it/s]

...

Lớp 1: 74.06%
Lớp 2: 25.94%
Lớp 1: 27.15%
Lớp 2: 72.85%
evidence_temp:--  gueorguiev phép trò_chuyện phút mỗi
Lớp 1: 27.29%
Lớp 2: 72.71%
evidence_temp:--  thử_thách gueorguiev
Lớp 1: 69.49%
Lớp 2: 30.51%
Lớp 1: 56.93%
Lớp 2: 43.07%
Lớp 1: 68.37%
Lớp 2: 31.63%
Lớp 1: 77.20%
Lớp 2: 22.80%

 10840
claim: -- gueorguiev phép trò_chuyện thử_thách
evidence_Predicted: -- gueorguiev phép trò_chuyện phút mỗi
Lớp 1: 76.93%
Lớp 2: 23.07%
Lớp 1: 71.83%
Lớp 2: 28.17%
Lớp 1: 31.63%
Lớp 2: 68.37%
evidence_temp:--  toàn tỉnh gia_lai ha mía
Lớp 1: 71.54%
Lớp 2: 28.46%
Lớp 1: 34.22%
Lớp 2: 65.78%
evidence_temp:--  cháy bất_thường
Lớp 1: 69.62%
Lớp 2: 30.38%
Lớp 1: 67.33%
Lớp 2: 32.67%
Lớp 1: 72.09%
Lớp 2: 27.91%
Lớp 1: 58.76%
Lớp 2: 41.24%
Lớp 1: 33.63%
Lớp 2: 66.37%
evidence_temp:--  giá mía cháy
Lớp 1: 73.23%
Lớp 2: 26.77%
Lớp 1: 77.95%
Lớp 2: 22.05%
Lớp 1: 52.19%
Lớp 2: 47.81%
Lớp 1: 77.48%
Lớp 2: 22.52%
Lớp 1: 77.81%
Lớp 2: 22.19%


100%|█████████▉| 999/1000 [08:51<00:00,  2.29it/s]

Lớp 1: 75.36%
Lớp 2: 24.64%
Lớp 1: 77.88%
Lớp 2: 22.12%
Lớp 1: 77.54%
Lớp 2: 22.46%
Lớp 1: 73.39%
Lớp 2: 26.61%
Lớp 1: 77.85%
Lớp 2: 22.15%
Lớp 1: 76.92%
Lớp 2: 23.08%

 15809
claim: -- tỉnh gia_lai huy_động người_dân cân_nhắc trồng mía dày diện_tích mía
evidence_Predicted: -- toàn tỉnh gia_lai ha mía
Lớp 1: 76.91%
Lớp 2: 23.09%
Lớp 1: 30.25%
Lớp 2: 69.75%
evidence_temp:--  tuy_nhiên hiện xác_định hướng đi
Lớp 1: 27.82%
Lớp 2: 72.18%
evidence_temp:--  mơ_hồ ngành
Lớp 1: 78.24%
Lớp 2: 21.76%


100%|██████████| 1000/1000 [08:51<00:00,  1.88it/s]

Lớp 1: 34.35%
Lớp 2: 65.65%
evidence_temp:--  không tương_lai liệu ngành kinh_tế hay kỹ_thuật việc_làm tốt
Lớp 1: 69.07%
Lớp 2: 30.93%
Lớp 1: 69.84%
Lớp 2: 30.16%

 881
claim: -- mơ_hồ lương ngành kinh_tế
evidence_Predicted: -- tuy_nhiên hiện xác_định hướng đi


# Lưu kết quả

In [24]:
import os
import json

def save_result(PATH, result_dict):
    # Tạo thư mục nếu nó chưa tồn tại
    os.makedirs(PATH, exist_ok=True)
    # Lưu từ điển vào tệp JSON
    result_path = os.path.join(PATH, "result_warmup_rationale.json")

    with open(result_path, "w", encoding="utf-8") as file:
        json.dump(converted_data, file, ensure_ascii=False)

In [25]:
save_path = "/content/drive/MyDrive/pRoBERTa/source/result"
save_result(save_path, converted_data)

In [ ]:
print("hello")